#### garrett r peternel | senior data scientist | lm

https://docs.aws.amazon.com/sagemaker/latest/dg/deepar.html
    
https://aws.amazon.com/blogs/machine-learning/now-available-in-amazon-sagemaker-deepar-algorithm-for-more-accurate-time-series-forecasting/

# aws sagemaker deepAr

In [ ]:
print("="*100)

# mods

In [ ]:
%matplotlib inline

import sys
from urllib.request import urlretrieve
import zipfile
from dateutil.parser import parse
import json
from random import shuffle
import random
import datetime
import os

import boto3
import s3fs
import sagemaker
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt

from __future__ import print_function
from ipywidgets import interact, interactive, fixed, interact_manual
import ipywidgets as widgets
from ipywidgets import IntSlider, FloatSlider, Checkbox

# sagemaker integration

In [ ]:
sagemaker_session = sagemaker.Session()
region = sagemaker_session.boto_region_name
role = sagemaker.get_execution_role()

s3_bucket = 'aero-iiot'  
s3_prefix = 'datasets/e363549/sagemaker/sgMkr-deepAr-Fad2'
data_key = 'datasets/e363549/queries/fad2_full.csv.gz'

s3_data_path = "s3://{}/{}/data".format(s3_bucket, s3_prefix)
s3_output_path = "s3://{}/{}/output".format(s3_bucket, s3_prefix)

# algs

In [ ]:
image_name = sagemaker\
.amazon\
.amazon_estimator.get_image_uri(region, "forecasting-deepar", "latest")

# read device data

In [ ]:
def read(data_key, s3_bucket):
    data_location = 's3://{}/{}'.format(s3_bucket, data_key)
    df = pd.read_csv(data_location,\
                     compression="gzip",\
                     sep=",",\
                     index_col=0,\
                     parse_dates=True)
    return df

In [ ]:
raw_df = read(data_key, s3_bucket)

# model cleaning

In [ ]:
def ad_hoc_clean(df):
    df = df.select_dtypes(exclude=['object'])
    df = df.resample("1min").last().ffill()
    df = df.dropna(axis=1, how='all')
    df = df.drop(columns=df.columns[df.nunique()==1])
    return df

In [ ]:
agg_df = ad_hoc_clean(raw_df)

# scale data

In [ ]:
from sklearn.preprocessing import StandardScaler

def scales(df):
    scaler = StandardScaler()
    df[df.columns] = scaler.fit_transform(df[df.columns])
    return df

In [ ]:
scales_df = scales(agg_df)

In [ ]:
scales_df.head()

# time series sequence

In [ ]:
def ts(df):
    num_timeseries = df.shape[1]
    time_series = []
    for i in range(num_timeseries):
        time_series.append(df.iloc[:,i])
    return time_series

In [ ]:
time_series = ts(agg_df)

In [ ]:
len(time_series)

In [ ]:
fig, axs = plt.subplots(5, 2, figsize=(20, 20), sharex=True)
axx = axs.ravel()
for i in range(0, 10):
    time_series[i].loc["2019-10-01":"2019-10-14"].plot(ax=axx[i])
    axx[i].set_xlabel("date")    
    axx[i].set_ylabel("feature")   
    axx[i].grid(which='minor', axis='x')

# deepAr ts params

In [ ]:
freq = '1min'
prediction_length = 10
context_length = 10

# train time series sequence

In [ ]:
def train_ts(l, prediction_length):
    time_series_training = []
    for ts in time_series:
        time_series_training.append(ts[:-prediction_length])
    return time_series_training

In [ ]:
time_series_training = train_ts(time_series, prediction_length)

In [ ]:
time_series[0].plot(label='test')
time_series_training[0].plot(label='train', ls=':')
plt.legend()
plt.show()

# important!

In [ ]:
print('remove the trailing 10 points per each time series to define the training set: {}'\
      .format(len(time_series_training[0])))
print('the test set contains the full sequence of each time series: {}'\
      .format(len(time_series[0])))

# required json string format

In [ ]:
def series_to_obj(ts, cat=None):
    obj = {"start": str(ts.index[0]), "target": list(ts)}
    if cat is not None:
        obj["cat"] = cat
    return obj

def series_to_jsonline(ts, cat=None):
    return json.dumps(series_to_obj(ts, cat))

In [ ]:
encoding = "utf-8"
s3filesystem = s3fs.S3FileSystem()

with s3filesystem.open(s3_data_path + "/train/train.json", 'wb') as fp:
    for ts in time_series_training:
        fp.write(series_to_jsonline(ts).encode(encoding))
        fp.write('\n'.encode(encoding))

with s3filesystem.open(s3_data_path + "/test/test.json", 'wb') as fp:
    for ts in time_series:
        fp.write(series_to_jsonline(ts).encode(encoding))
        fp.write('\n'.encode(encoding))

# estimator

In [ ]:
estimator = sagemaker.estimator.Estimator(
    sagemaker_session=sagemaker_session,
    image_name=image_name,
    role=role,
    train_instance_count=1,
    train_instance_type='ml.p3.2xlarge',
    base_job_name='sgMkr-deepAr-Fad2',
    output_path=s3_output_path
)

# tuners

In [ ]:
hyperparameters = {
    "time_freq": freq,
    "context_length": str(context_length),
    "prediction_length": str(prediction_length),
    "num_cells": "40",
    "num_layers": "3",
    "likelihood": "student-T",
    "epochs": "100",
    "mini_batch_size": "128",
    "learning_rate": "0.001",
    "dropout_rate": "0.05",
    "early_stopping_patience": "10"
}

In [ ]:
estimator.set_hyperparameters(**hyperparameters)

# fit

In [ ]:
data_channels = {
    "train": "{}/train/".format(s3_data_path),
    "test": "{}/test/".format(s3_data_path)
}

estimator.fit(inputs=data_channels, wait=True)

# endpoint

In [ ]:

job_name = estimator.latest_training_job.name

endpoint_name = sagemaker_session.endpoint_from_job(
    job_name=job_name,
    initial_instance_count=8,
    instance_type='ml.m4.xlarge',
    deployment_image=image_name,
    role=role
)

In [ ]:
predictor.delete_endpoint()

# predictor

In [ ]:
class DeepARPredictor(sagemaker.predictor.RealTimePredictor):
    
    def __init__(self, *args, **kwargs):
        super().__init__(*args, content_type=sagemaker.content_types.CONTENT_TYPE_JSON, **kwargs)
    
    def set_prediction_parameters(self, freq, prediction_length):
        """Set the time frequency and prediction length parameters. This method **must** be called
        before being able to use `predict`.
        
        Parameters:
        freq -- string indicating the time frequency
        prediction_length -- integer, number of predicted time points
        
        Return value: none.
        """
        self.freq = freq
        self.prediction_length = prediction_length

    def predict(self, ts, cat=None, dynamic_feat=None, 
                num_samples=100, return_samples=False, quantiles=["0.1", "0.5", "0.9"]):
        """Requests the prediction of for the time series listed in `ts`, each with the (optional)
        corresponding category listed in `cat`.
        
        ts -- `pandas.Series` object, the time series to predict
        cat -- integer, the group associated to the time series (default: None)
        num_samples -- integer, number of samples to compute at prediction time (default: 100)
        return_samples -- boolean indicating whether to include samples in the response (default: False)
        quantiles -- list of strings specifying the quantiles to compute (default: ["0.1", "0.5", "0.9"])
        
        Return value: list of `pandas.DataFrame` objects, each containing the predictions
        """
        prediction_time = ts.index[-1] + 1
        quantiles = [str(q) for q in quantiles]
        req = self.__encode_request(ts, cat, dynamic_feat, num_samples, return_samples, quantiles)
        res = super(DeepARPredictor, self).predict(req)
        return self.__decode_response(res, ts.index.freq, prediction_time, return_samples)
    
    def __encode_request(self, ts, cat, dynamic_feat, num_samples, return_samples, quantiles):
        instance = series_to_dict(ts, cat if cat is not None else None, dynamic_feat if dynamic_feat else None)

        configuration = {
            "num_samples": num_samples,
            "output_types": ["quantiles", "samples"] if return_samples else ["quantiles"],
            "quantiles": quantiles
        }
        
        http_request_data = {
            "instances": [instance],
            "configuration": configuration
        }
        
        return json.dumps(http_request_data).encode('utf-8')
    
    def __decode_response(self, response, freq, prediction_time, return_samples):
        # we only sent one time series so we only receive one in return
        # however, if possible one will pass multiple time series as predictions will then be faster
        predictions = json.loads(response.decode('utf-8'))['predictions'][0]
        prediction_length = len(next(iter(predictions['quantiles'].values())))
        prediction_index = pd.DatetimeIndex(start=prediction_time, freq=freq, periods=prediction_length)        
        if return_samples:
            dict_of_samples = {'sample_' + str(i): s for i, s in enumerate(predictions['samples'])}
        else:
            dict_of_samples = {}
        return pd.DataFrame(data={**predictions['quantiles'], **dict_of_samples}, index=prediction_index)

    def set_frequency(self, freq):
        self.freq = freq
        
def encode_target(ts):
    return [x if np.isfinite(x) else "NaN" for x in ts]        

def series_to_dict(ts, cat=None, dynamic_feat=None):
    """Given a pandas.Series object, returns a dictionary encoding the time series.

    ts -- a pands.Series object with the target time series
    cat -- an integer indicating the time series category

    Return value: a dictionary
    """
    obj = {"start": str(ts.index[0]), "target": encode_target(ts)}
    if cat is not None:
        obj["cat"] = cat
    if dynamic_feat is not None:
        obj["dynamic_feat"] = dynamic_feat        
    return obj

In [ ]:
predictor = DeepARPredictor(
    endpoint=endpoint_name,
    sagemaker_session=sagemaker_session
)
predictor.set_prediction_parameters(freq, prediction_length)

# predict

In [ ]:
predictor.predict(ts=time_series[0], quantiles=[0.10, 0.5, 0.90]).head(10)

#### garrett r peternel | senior data scientist | lm